In [1]:
!pip install telepot
!pip install geocoder

In [2]:
import cv2
import numpy as np
import time
import geocoder
from collections import deque
from tensorflow.keras.models import load_model
import telepot
import firebase_admin
from firebase_admin import credentials, db

# Initialize Firebase Admin SDK with your credentials
cred = credentials.Certificate("./serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://rvidastech-default-rtdb.firebaseio.com/'
})

# Initialize Telegram bot
bot = telepot.Bot('7186069023:AAFg_KT_mSUd5ofySXlwViyK4k32Gu3ahnU')

def get_current_location():
    location = geocoder.ip('me')
    location_info = f"LOCATION: {location.city}, {location.state}, {location.country}\n"
    return location_info

def store_data_to_firebase(location_info, timeMoment, video_filename, filename):
    ref = db.reference('violence_alerts')
    ref.push({
        'location': location_info,
        'time': timeMoment,
        'video_url': video_filename,
        'image_url': filename
    })

def send_telegram_alert(location_info, timeMoment, video_filename, filename):
    bot.sendMessage(-1001991802221, f"VIOLENCE ALERT!!\n{location_info}TIME: {timeMoment}")
    with open(video_filename, 'rb') as video_file:
        bot.sendVideo(-1001991802221, video_file)
    bot.sendPhoto(-1001991802221, photo=open(filename, 'rb'))

def print_results():
    filename = 'savedImage.jpg'
    video_filename = 'recordedVideo.avi'

    # Get current location
    location_info = get_current_location()

    # Load pre-trained Keras model
    print("Loading model ...")
    model = load_model('./modelnew.h5')

    # Recompile the model if needed
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Capture video from webcam
    vs = cv2.VideoCapture(0)

    # Check if webcam is opened successfully
    if not vs.isOpened():
        print("Error: Failed to open webcam.")
        return

    # Get webcam dimensions
    W = int(vs.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(video_filename, fourcc, 30, (W, H), True)

    # Loop through video frames
    while True:
        grabbed, frame = vs.read()

        if not grabbed:
            break

        output = frame.copy()

        # Preprocess frame for model input
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(1, 128, 128, 3) / 255.0  # Add an extra dimension for batch

        # Predict violence probability
        preds = model.predict(frame)[0]
        label = (preds > 0.50)  # Thresholding predictions

        # Adding text to the output frame
        text = "Violence: {}".format(label)
        text_color = (0, 0, 255) if label else (0, 255, 0)
        cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, text_color, 3)

        # Write frame to output video
        writer.write(output)

        # Display output frame
        cv2.imshow("Output", output)

        # Save image and send alert if violence detected
        if label:
            timeMoment = time.strftime("%H:%M:%S \nDate %d-%m-%y")
            store_data_to_firebase(location_info, timeMoment, video_filename, filename)
            send_telegram_alert(location_info, timeMoment, video_filename, filename)

        # Check for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    # Clean up
    print("[INFO] cleaning up...")
    if writer is not None:
        writer.release()
    vs.release()
    cv2.destroyAllWindows()

print_results()


Loading model ...


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━

C:\Users\Manjit Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\telepot\__init__.py:1074: SyntaxWarning: invalid escape sequence '\s'
  """


KeyboardInterrupt: 